In [1]:
from main import RFest
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import time

In [34]:
#Checking number of cores
estimator=RFest(drop_rate=0.99999)

In [ ]:
estimator.rows_range

In [27]:
df=estimator.generate_data()

2018-09-01 16:35:01,766 - main - Generating dummy training durations to create a training set


In [31]:
df[0].num_rows.describe()

count     7973.000000
mean      3657.782516
std       4453.865502
min        100.000000
25%        100.000000
50%       1000.000000
75%      10000.000000
max      10000.000000
Name: num_rows, dtype: float64

In [3]:
estimator.model_fit()

2018-08-29 13:27:43,756 - main - Generating dummy training durations to create a training set
2018-08-29 13:34:21,884 - main - Fitting LR to estimate training durations
2018-08-29 13:34:22,011 - main - Saving LR to LR_estimator.pkl
2018-08-29 13:34:22,017 - main - R squared on train set is 0.28951561653299607
2018-08-29 13:34:22,022 - main - MAPE on train set is: 103.06569535701071
2018-08-29 13:34:22,026 - main - MAPE on test set is: 110.35796054230411
2018-08-29 13:34:22,032 - main - MSE on train set is 0.00210308752680741
2018-08-29 13:34:22,039 - main - MSE on test set is 0.0021832059801950346


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [4]:
rf= RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=5, max_leaf_nodes=5, min_impurity_decrease=0.0,
           min_impurity_split=5, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


In [15]:
estimator.estimate_duration(np.random.rand(20000000,10),rf)


2018-08-29 13:46:35,135 - main - Fetching estimator: LR_estimator.pkl
2018-08-29 13:46:35,143 - main - Training your model should take ~ 22.0532326164 seconds


array([ 22.05323262])

In [16]:
X,y=np.random.rand(20000000,10),np.random.rand(20000000,1)
start_time = time.time()
rf.fit(X,y)
elapsed_time = time.time() - start_time

In [17]:
elapsed_time

49.866721868515015